In [1]:
%matplotlib inline

# Imports

In [2]:
###Standard imports
import pandas as pd
from collections import Counter
from scipy import stats

###Slightly specialized
from Bio import SeqIO
from Bio.Data import CodonTable

###Really specialized
import sys
sys.path.append('../../iCUB/')
import iCUB

# Define constants for genome extraction

In [3]:
genome = list(SeqIO.parse('../Data/ecoli_info/NC_000913.3.gb', 'genbank'))
assert len(genome) == 1
genome = genome[0]

length_of_upstream_seq = 30

# Establish a general dataframe describing various coding sequence fesatures from the genome

In [4]:
loci = []
gene_ids = []
cds_seqs = []
us_seqs = []
starts = []
stops = []
strands = []

for feature in genome.features:
    nt_seq_len = len(str(genome.seq))
    if feature.type == 'CDS': ###Caring only about protein coding sequences aka CDS
        try:
            locus_id = feature.qualifiers['locus_tag'][0]
        except:
            locus_id = ''
        try:
            gene_id = feature.qualifiers['gene'][0]
        except:
            gene_id = ''
        start = feature.location.start
        stop = feature.location.end
        strand = feature.location.strand
        assert stop > start
        if strand == 1:
            strand = '+'
        elif strand == -1:
            strand = '-'
        else:
            print('ERROR in strand')
            break
        if strand == '-':
            if stop + 30 > nt_seq_len: ###Case that may occur at chromosome end
                continue
            cds_seq = str(genome.seq[start:stop].reverse_complement())
            us_seq = str(genome.seq[stop:stop+length_of_upstream_seq].reverse_complement())
        elif strand == '+':
            if start < 30: ###Case that may occur at chromosome beginning
                continue
            cds_seq = str(genome.seq[start:stop])
            us_seq = str(genome.seq[start-length_of_upstream_seq:start])
        ###Add everyone to growing lists because it's a faster way to set up dataframes
        loci.append(locus_id)
        gene_ids.append(gene_id)
        starts.append(start)
        stops.append(stop)
        strands.append(strand)
        cds_seqs.append(cds_seq)
        us_seqs.append(us_seq)

In [5]:
df = pd.DataFrame(zip(loci, gene_ids, starts, stops, strands, cds_seqs, us_seqs)) 
df.columns = ['locus_tag', 'gene', 'start_loc', 'stop_loc', 'strand', 'cds_seq', 'us_seq']
print(df.shape)
df.head()

(4357, 7)


,locus_tag,gene,start_loc,stop_loc,strand,cds_seq,us_seq
0,b0001,thrL,189,255,+,ATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCA...,CAGATAAAAATTACAGAGTACACAACATCC
1,b0002,thrA,336,2799,+,ATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAAC...,TTTTCGACCAAAGGTAACGAGGTAACAACC
2,b0003,thrB,2800,3733,+,ATGGTTAAAGTTTATGCCCCGGCTTCCAGTGCCAATATGAGCGTCG...,GTACCCTCTCATGGAAGTTAGGAGTCTGAC
3,b0004,thrC,3733,5020,+,ATGAAACTCTACAATCTGAAAGATCACAACGAGCAGGTCAGCTTTG...,ACGGCGGGCGCACGAGTACTGGAAAACTAA
4,b0005,yaaX,5233,5530,+,GTGAAAAAGATGCAATCTATCGTACTCGCACTTTCCCTGGTTCTGG...,CATAACGGGCAATGATAAAAGGAGTAACCT


In [6]:
###Test that I extracted upstream sequences properly
print(length_of_upstream_seq)
print(Counter(list(df['us_seq'].str.len())))

30
Counter({30: 4357})


In [7]:
df['cds_len'] = df['stop_loc'] - df['start_loc']
assert all(df['cds_len'] == df['cds_seq'].str.len())
print('Max and min cds lengths:', df['cds_len'].max(), '\t', df['cds_len'].min())

Max and min cds lengths: 8622 	 42


**Keep track of genes that will and won't be useful for downstream analyses**

The only addition at the end should be a `well_behaved` column

In [8]:
print(df.shape)
###First ensure that locus tags are unique and drop duplicates entirely
df = df.drop_duplicates('locus_tag')

###Next set up a simple column that tracks the suitability of each gene
df['well_behaved'] = True

###Ensure that the coding sequence is a multiple of 3 (otherwise, what is happening?)
df.at[df[df['cds_seq'].str.len()%3 != 0].index, 'well_behaved'] = False

###Count the number of canonical bases
for letter in ['A', 'T', 'G', 'C']:
    df[letter] = df['cds_seq'].str.count(letter)
df['ATGC'] = df[['A', 'T', 'G', 'C']].sum(axis=1)
# ###And ensure that this number equals the coding sequence length
df.at[df[(df['stop_loc'] - df['start_loc']) != df['ATGC']].index, 'well_behaved'] = False

# ###Repeat the above procedure for upstream sequence regions
for letter in ['A', 'T', 'G', 'C']:
    df[letter] = df['us_seq'].str.count(letter)
df['ATGC'] = df[['A', 'T', 'G', 'C']].sum(axis=1)
###And ensure that this number equals the predefined upstream sequence length
df.at[df[df['ATGC'] != length_of_upstream_seq].index, 'well_behaved'] = False

###Remove those temporary columns
df = df.drop(columns=['A', 'T', 'G', 'C', 'ATGC'])
print(df.shape)

(4357, 8)
(4355, 9)


**Write a set of well-behaved coding sequences to run through ROC-SEMPPR (associated R code)**

In [9]:
with open('../Data/ecoli_info/NC_000913.3.CLEAN.CDS.fasta', 'w') as outfile:
    for index in df[df['well_behaved']==True].index:
        outfile.write('>{}\n'.format(df.loc[index]['locus_tag']))
        outfile.write('{}\n'.format(df.loc[index]['cds_seq']))

# Add in GC percent data

In [10]:
###Count the number of canonical bases
for letter in ['A', 'T', 'G', 'C']:
    df[letter] = df['cds_seq'].str.count(letter)
df['GC_percent_cds'] = df[['G', 'C']].sum(axis=1)/df[['A', 'T', 'G', 'C']].sum(axis=1)
df = df.drop(columns=['A', 'T', 'G', 'C'])
print(df.shape)

(4355, 10)


# Add the `ROC-SEMPPR` data 

(After running the `R` code offline)

`ROC-SEMPPR` is defined here:
https://academic.oup.com/gbe/article/7/6/1559/2465799

It is a pretty novel, and in my opinion under-cited / under-used, metric of codon usage bias that is rooted in population genetic theory. The calculations take some time to run, but require only the **set** of all coding sequences in question from a particular genome, with no other *a priori* information necessary. In this sense it's slightly more complex than `iCUB` but uses far less information than many other models that purport to predict gene expression from sequence information.

In [11]:
rocsemppr_df = pd.read_csv('../Data/roc_semppr_first_pass.csv')
rocsemppr_df = rocsemppr_df.drop(columns=rocsemppr_df.columns[2:])
rocsemppr_df.columns = ['locus_tag', 'roc_semppr_mean']
df = df.merge(rocsemppr_df, on='locus_tag', how='outer')
print(df.shape)
df.head()

(4355, 11)


,locus_tag,gene,start_loc,stop_loc,strand,cds_seq,us_seq,cds_len,well_behaved,GC_percent_cds,roc_semppr_mean
0,b0001,thrL,189,255,+,ATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCA...,CAGATAAAAATTACAGAGTACACAACATCC,66,True,0.515152,1.244106
1,b0002,thrA,336,2799,+,ATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAAC...,TTTTCGACCAAAGGTAACGAGGTAACAACC,2463,True,0.530654,1.034078
2,b0003,thrB,2800,3733,+,ATGGTTAAAGTTTATGCCCCGGCTTCCAGTGCCAATATGAGCGTCG...,GTACCCTCTCATGGAAGTTAGGAGTCTGAC,933,True,0.562701,0.994168
3,b0004,thrC,3733,5020,+,ATGAAACTCTACAATCTGAAAGATCACAACGAGCAGGTCAGCTTTG...,ACGGCGGGCGCACGAGTACTGGAAAACTAA,1287,True,0.528361,1.176750
4,b0005,yaaX,5233,5530,+,GTGAAAAAGATGCAATCTATCGTACTCGCACTTTCCCTGGTTCTGG...,CATAACGGGCAATGATAAAAGGAGTAACCT,297,True,0.538721,0.837528


# Add in `iCUB`

`iCUB` is a simple codon usage bias metric that was originally designed/implemented here:
https://royalsocietypublishing.org/doi/full/10.1098/rsif.2017.0667

It is of a class of codon usage bias metrics that simply calculate "disorder" from a single coding sequence with no other *a priori* information given (including knowledge of any other values in the genome in question). This class of metrics is best known for the `Effective Number of Codons`, but many subsequent works have illustrated the limitations of this basic metric and have proposed various improvements and variants. `iCUB` appears to out-perform all known metrics of this simple class.

In [12]:
icub_vals = []
for index in df[df['well_behaved']==True].index:
    cds_seq = str(df.loc[index]['cds_seq'])
    icub_vals.append(iCUB.iCUB_Calculator(cds_seq).get_iCUB())
    
df.at[df[df['well_behaved']==True].index, 'iCUB'] = icub_vals

# Add in the Codon Adaptation Index (`CAI`)

The `Codon Adaptation Index` is one of the most famous codon usage bias metrics, originally proposed here:
https://academic.oup.com/nar/article-abstract/15/3/1281/1166844

It has a crazy number of citations and generally performs well at the task of predicting gene expression from sequence information. However, it requires some arbitrary and extraneous information. At its heart, the metric is a **distance** metric. It measures the distance of a given coding sequence (in terms of its codon usage bias) to a **pre-defined reference set of coding sequences** that are presumed to be highly expressed, and from which codon-level weights are calculated. Some work has been done to try to choose that reference set in an automated fashion from only knowledge in the genome, but to my knowledge nothing seems to outperform the pre-defined reference sets proposed in the paper (used below). And of course, this is to be expected since that reference set is a bit of a cheat since it already requires some outside gene expression knowledge and folds this knowledge into predictions of gene expression.

In [13]:
def get_codon_dicts(n=11):
    """
    Gets the codon to amino acid look up dictionary and the amino acid to 
    (list of) codons dictionary using Biopython's CodonTable module
    """
    codon_to_aa = CodonTable.unambiguous_dna_by_id[n].forward_table
    aa_to_codons = {}
    for codon, aa in codon_to_aa.items():
        try:
            aa_to_codons[aa].append(codon)
        except:
            aa_to_codons[aa] = [codon]
    return codon_to_aa, aa_to_codons 
    
def flatten_lol(lol):
    """
    Merges a list of lists into a single large list
    """
    return [inner for outer in lol for inner in outer]

def tripletize(cds_string):
    """
    Split a coding sequence into triplets (codons)
    """
    assert len(cds_string) % 3 == 0
    return [cds_string[i:i+3] for i in range(0, len(cds_string), 3)]

def calculate_RSCUs(codon_count_dict):
    """
    Calculates the relative synonymous codon usage values for a *SINGLE* dictionary
    containing codon counts (codon:count) for *ONE* amino acid
    """
    n_total = sum(codon_count_dict.values())
    unique_codons = len(codon_count_dict.keys())
    rscu_dict = {}  
    if n_total != 0:
        for codon, codon_count in codon_count_dict.items():
            rscu_dict[codon] = codon_count / (float(n_total) / unique_codons)
    else:
        for codon, codon_count in codon_count_dict.items():
            rscu_dict[codon] = 0 
    return rscu_dict

def calculate_codon_weights(list_of_cds_strings, zero_val=0.001):
    """
    Calulates the weight of each codon from a reference set of genes
    """
    ###Get housekeeping dictionaries
    codon_to_aa, aa_to_codons = get_codon_dicts()
    
    if len(list_of_cds_strings) > 1: #when given a list of strings, make it one long list of codons
        all_codons = []
        for cds in list_of_cds_strings:
            all_codons.append(tripletize(cds))
        all_codons = flatten_lol(all_codons)
    else:
        all_codons = tripletize(list_of_cds_strings)

    all_codon_counts = Counter(all_codons) #create dictionary of codon counts in the set
    
    ###Workhorse of the code
    weights_dict = {}
    for aa, codons in aa_to_codons.items(): #iterates through single amino acids
        single_aa_codon_counts = {k: all_codon_counts.get(k, None) for k in codons}
        single_rscu_dict = calculate_RSCUs(single_aa_codon_counts) #get RSCU dictionary for the aa
        max_rscu = max(single_rscu_dict.values()) #weights are normalized by the max RSCU
        for codon, rscu_val in single_rscu_dict.items():
            if single_rscu_dict[codon] == 0:
                weights_dict[codon] = zero_val
            else:
                weights_dict[codon] = rscu_val / max_rscu
    return weights_dict

def calculate_cai(single_cds_str, weights_dict):
    """
    Calculates the CAI from a gene (string) and a dictionary of codon weight values.
    """
    codon_list = tripletize(single_cds_str)
    weights_list = []
    for codon in codon_list:
        weights_list.append(weights_dict.get(codon, None))
    weights_list =  [i for i in weights_list if i] 
    cai = stats.gmean(weights_list)
    return cai

**The CAI relies on a reference set of sequences, previously defined, to determine individual codon weight values. I have taken these from the original 1987 implementation as described for *E. coli***

In [14]:
reference_set_loci = pd.read_csv('../Data/ecoli_info/original_CAI_refset.txt', header=None, names=['locus_tag'])
ref_set_seqs = list(df.merge(reference_set_loci, on='locus_tag')['cds_seq'])
cai_weights_dict = calculate_codon_weights(ref_set_seqs)

In [15]:
cai_vals = []
for index in df[df['well_behaved']==True].index:
    cds_seq = df.loc[index]['cds_seq']
    cai_vals.append(calculate_cai(cds_seq, cai_weights_dict))
    
df.at[df[df['well_behaved']==True].index, 'CAI'] = cai_vals

# Add in knowledge of the `tAI` 

The so-called `tRNA adaptation index` is a very orthogonal metric that uses knowledge of the tRNA genes contained within the genome of interest to calculate a coding sequence specific metric of adaptation to this hypothetical tRNA pool. To my knowledge it was originally proposed here:
https://academic.oup.com/nar/article/32/17/5036/1333956

And I am taking the original codon weight values for *E. coli* from here:
https://www.pnas.org/content/107/8/3645

There was a subsequent re-implementation / variation that I'm also taking data from:
https://academic.oup.com/bioinformatics/article/33/4/589/2593585

But since that method is woefully inadequate in terms of its accessibility and usability (matlab?!), I'm taking the actual codon-level weights for this variant from yet another publication that (as best as I can tell) simply re-implemented the same basic model with more transparency:
https://genominfo.org/journal/view.php?doi=10.5808/GI.2018.16.4.e28

This is all a long wind up to say that I should probably re-implement the weight calculations from scratch one day. 

In [16]:
weights_df1 = pd.read_csv('../Data/ecoli_info/NC_000913.3.original_tAI_weights.csv', header=None)
weights_df1 = weights_df1[weights_df1[1].isnull()==False]
weights_df1.columns = ['codon', 'value']
weights_dict1 = pd.Series(weights_df1['value'].values,index=weights_df1['codon']).to_dict()
#
weights_df2 = pd.read_csv('../Data/ecoli_info/NC_000913.3.stAIcalc_weights.csv', index_col=0)
weights_df2.columns = ['codon', 'value']
weights_dict2 = pd.Series(weights_df2['value'].values,index=weights_df2['codon']).to_dict()

In [17]:
tai_vals1 = []
tai_vals2 = []
for index in df[df['well_behaved']==True].index:
    cds_seq = df.loc[index]['cds_seq']
    codons = tripletize(str(cds_seq))
    tempy1 = []
    tempy2 = []
    for codon in codons:
        tempy1.append(weights_dict1.get(codon, None))
        tempy2.append(weights_dict2.get(codon, None))
    tempy1 = [i for i in tempy1 if i]
    tempy2 = [i for i in tempy2 if i]
    tai_vals1.append(stats.gmean(tempy1))
    tai_vals2.append(stats.gmean(tempy2))

df.at[df[df['well_behaved']==True].index, 'tAI'] = tai_vals1
df.at[df[df['well_behaved']==True].index, 'stAIcalc'] = tai_vals2

# TO DO: Add in Shine-Dalgarno sequence strength, mRNA secondary structure, and (possibly) dN/dS metrics

In [18]:
import json
import subprocess

def add_RBS_energy(df, energy_dict, col_name='aSD_binding',\
                   gaps=(4,10), expected_len=30, RBS_len=6):
    '''
    This function adds a new column to the df based off of pre-computed free energy 
    vals (created with RNAcofold and stored in the corresponding energy_dict). 
    
    Inputs:
        df - 
        energy_dict - 
        gaps - 
        expected_len - 
    
    Outputs:
        df - the transformed df object now containing the new column
    '''
    for index in df.index:
        upstream = df.loc[index,'us_seq']
        test_string = upstream.replace('T', 'U')
        ###Ensure that the sequence is the proper expected length
        if len(test_string) != expected_len:
            continue
        ###Ensure that the sequence has no abnormal bases
        if test_string.count('A') + test_string.count('U') +\
                                    test_string.count('C') + test_string.count('G') != expected_len:
            continue
            
        ###Calculate the energy for the indicated gap offsets
        energy_list = []
        for gap in range(gaps[0],gaps[1]+1): ###Doing some complicated indexing to get proper gaps
             energy_list.append(energy_dict[test_string[-gap - RBS_len : -gap]])

        df.at[index, col_name] = min(energy_list)        
    return df


def call_RNAfold(sequence):
    sequence = sequence.replace('T', 'U')
    MyOut = subprocess.Popen(['RNAfold', '-p', '--noPS', '--noDP', '--constraint'],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE, 
            stderr=subprocess.STDOUT)
    stdout, stderr = MyOut.communicate(input=str.encode(sequence))
    return stdout

def get_energy_RNAfold(stdout_string):
    '''
    '''
    temp = stdout_string.decode('utf-8') 
    mfe_line = temp.split('\n')[-5]
    mfe_val = mfe_line[mfe_line.index(' '):]
    mfe_val = mfe_val.strip().strip('()').strip()
    #
    ensemble_line = temp.split('\n')[-4]
    ensemble_val = ensemble_line[ensemble_line.index(' '):]
    ensemble_val = ensemble_val.strip().strip('[]').strip()
    return float(ensemble_val), float(mfe_val)    

def add_secondary_structure(df):
    '''
    
    '''
    for index in df.index:
        us_seq = df.loc[index]['us_seq'] 
        cds_seq = df.loc[index]['cds_seq']
        if len(cds_seq) < 90:
            continue
        beg_cds_seq = cds_seq[:60]
        if len(us_seq) == 30 and len(beg_cds_seq) == 60:
            seq = us_seq + beg_cds_seq
            rna_out = call_RNAfold(seq)
            e1, e2 = get_energy_RNAfold(rna_out)
            df.at[index, 'sec_struct'] = e1
            #
            rna_out = call_RNAfold(seq+'\n'+ribo_bound_constraint)
            e1, e2 = get_energy_RNAfold(rna_out)
            df.at[index, 'sec_struct_bound'] = e1

    return df

In [19]:
sep = '\t'
upstream_len = 30

with open('../Data/energyRef_CCUCCU_ensemble_noneConstraint.json', 'r') as infile:
       energy_dict = json.load(infile)


ribo_bound_constraint = ('.'*16) + ('x'*28)+ ('.'*46)
assert len(ribo_bound_constraint)==90

In [20]:
###Adds the ribosome binding site energy column
df = add_RBS_energy(df, energy_dict, col_name='aSD_binding', gaps=(4,10))

###Adds the secondary structure column
df = add_secondary_structure(df)

###And a little manipulation
df['Start_accessibility'] = df['sec_struct'] - df['sec_struct_bound']

In [21]:
df.head()

,locus_tag,gene,start_loc,stop_loc,strand,cds_seq,us_seq,cds_len,well_behaved,GC_percent_cds,roc_semppr_mean,iCUB,CAI,tAI,stAIcalc,aSD_binding,sec_struct,sec_struct_bound,Start_accessibility
0,b0001,thrL,189,255,+,ATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCA...,CAGATAAAAATTACAGAGTACACAACATCC,66,True,0.515152,1.244106,32.046035,0.617266,0.262286,0.258417,-2.45,NaN,NaN,NaN
1,b0002,thrA,336,2799,+,ATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAAC...,TTTTCGACCAAAGGTAACGAGGTAACAACC,2463,True,0.530654,1.034078,55.949262,0.353246,0.230564,0.216862,-5.42,-21.31,-12.81,-8.50
2,b0003,thrB,2800,3733,+,ATGGTTAAAGTTTATGCCCCGGCTTCCAGTGCCAATATGAGCGTCG...,GTACCCTCTCATGGAAGTTAGGAGTCTGAC,933,True,0.562701,0.994168,56.062386,0.357812,0.216292,0.210420,-6.51,-21.87,-14.05,-7.82
3,b0004,thrC,3733,5020,+,ATGAAACTCTACAATCTGAAAGATCACAACGAGCAGGTCAGCTTTG...,ACGGCGGGCGCACGAGTACTGGAAAACTAA,1287,True,0.528361,1.176750,53.052776,0.394675,0.231407,0.209784,-3.40,-24.44,-20.71,-3.73
4,b0005,yaaX,5233,5530,+,GTGAAAAAGATGCAATCTATCGTACTCGCACTTTCCCTGGTTCTGG...,CATAACGGGCAATGATAAAAGGAGTAACCT,297,True,0.538721,0.837528,50.705530,0.374371,0.197715,0.208944,-6.51,-17.15,-7.38,-9.77


In [23]:
df.tail()

,locus_tag,gene,start_loc,stop_loc,strand,cds_seq,us_seq,cds_len,well_behaved,GC_percent_cds,roc_semppr_mean,iCUB,CAI,tAI,stAIcalc,aSD_binding,sec_struct,sec_struct_bound,Start_accessibility
4350,b4399,creC,4636695,4638120,+,ATGCGTATCGGCATGCGGTTGTTGCTGGGCTATTTTTTACTGGTGG...,CGGCATGGGATATAGCCTGAGGGGCCTGTA,1425,True,0.538947,0.829794,55.967103,0.293537,0.212672,0.206721,-6.09,-31.71,-18.33,-13.38
4351,b4400,creD,4638177,4639530,+,ATGTTGAAATCCCCCCTGTTCTGGAAAATGACTAGCCTGTTTGGTG...,TATCCTGCTGCCATTGCAAAGGAGAAGACT,1353,True,0.495935,0.739848,54.634345,0.288635,0.210078,0.199677,-7.00,-16.18,-10.40,-5.78
4352,b4401,arcA,4639589,4640306,-,ATGCAGACCCCGCACATTCTTATCGTTGAAGACGAGTTGGTAACAC...,TCGATTTAGTTGGCAATTTAGGTAGCAAAC,717,True,0.510460,1.676448,48.981781,0.530773,0.275348,0.239921,-2.60,-16.36,-9.19,-7.17
4353,b4402,yjjY,4640401,4640542,+,ATGACTAAAGTACGTAATTGCGTTCTTGATGCACTTTCCATCAACG...,CATGCTGCAAATTAACATGATCGGCGTAAC,141,True,0.425532,0.802032,47.451431,0.314394,0.219204,0.230201,-1.76,-14.24,-7.14,-7.10
4354,b4403,yjtD,4640941,4641628,+,ATGCGTATAACGATTATTCTGGTCGCACCCGCCAGAGCAGAAAATA...,CGCGGTTATTTGATCAAGAAGAGTGGCAAT,687,True,0.497817,0.566772,53.610186,0.261990,0.204571,0.202168,-2.65,-20.19,-14.11,-6.08


# Write data to a file

In [22]:
df.to_csv('../Data/ecoli_info/current_ecoli_master_table.tsv', sep='\t', index=False)

# Scratch

In [ ]:
temp_df = df[df['well_behaved']==True]
print(temp_df.shape)

In [ ]:
codon_metrics = ['roc_semppr_mean', 'CAI', 'tAI', 'stAIcalc', 'iCUB']
for i, metric_a in enumerate(codon_metrics):
    for metric_b in codon_metrics:
        print(metric_a, metric_b, '\t', stats.spearmanr(temp_df[metric_a], temp_df[metric_b]))